In [ ]:
import sys
import os
import uuid
import zipfile

# Add the path to the wind_fusion directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from wind_fusion.utils import fetch_cds

# Fetch wind power generation

In [ ]:
path = '../data/climate and energy indicators for europe/'
filename = f'wind_power_{str(uuid.uuid4())[:4]}.zip'
fetch_cds.fetch_wind_power_offshore(["2022"], [f'{i:02}' for i in range(1, 13)], path + filename)

# Unzip and delete zipfile
with zipfile.ZipFile(path + filename, 'r') as zip_ref:
    zip_ref.extractall(path)
os.remove(path + filename)

In [ ]:
from wind_fusion import energy_dataset
import xarray as xr
from typing import List


def _merge_datasets(filepaths: List[str]) -> xr.Dataset:
        ds = xr.Dataset()

        for filepath in filepaths:
            ds = xr.merge(
                [ds, xr.open_dataset(filepath)], compat="equals", join="inner"
            )
        return ds






# energy_wind_data = energy_dataset.EnergyDataset(["/home/hk-project-test-mlperf/om1434/masterarbeit/data/climate and energy indicators for europe/H_ERA5_ECMW_T639_WOF_0100m_Euro_025d_S202403010000_E202403312300_CFR_MAP_01h_NA-_noc_org_NA_NA---_NA---_PhM01.nc",
#                                                  "/lsdf/kit/imk-tro/projects/Gruppe_Quinting/ec.era5/1959-2023_01_10-wb13-6h-1440x721.zarr"])

merged = _merge_datasets(["/home/hk-project-test-mlperf/om1434/masterarbeit/data/climate and energy indicators for europe/H_ERA5_ECMW_T639_WOF_0100m_Euro_025d_S202403010000_E202403312300_CFR_MAP_01h_NA-_noc_org_NA_NA---_NA---_PhM01.nc",
                                                  "/lsdf/kit/imk-tro/projects/Gruppe_Quinting/ec.era5/1959-2023_01_10-wb13-6h-1440x721.zarr"])

In [ ]:
# energy_wind_data.get_dataset()
merged

In [ ]:
import os
from typing import List
path = "/home/hk-project-test-mlperf/om1434/masterarbeit/data/climate and energy indicators for europe"

def list_files(directory: str) -> List[str]:
    return [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

files = list_files(path)
files

In [ ]:
import xarray as xr
xr.concat([xr.open_dataset(f) for f in files], dim="time")

In [ ]:
for file in files:
    print(xr.open_dataset(file).time)

In [ ]:
import cdsapi
import time
import threading

dataset = "sis-energy-derived-reanalysis"
request = {
    # 'variable': ['wind_power_generation_offshore'],
    'variable': ["wind_power_generation_onshore"],
    'spatial_aggregation': ['original_grid'],
    'energy_product_type': ['capacity_factor_ratio'],
    'temporal_aggregation': ['hourly'],
    'year': None,
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
}


def retrieve_data(year):
    print(f"Starting job for year: {year}")
    request['year'] = year
    client = cdsapi.Client()
    client.retrieve(dataset, request)

threads = []
# for year in [str(y) for y in range(1979, 2024)]:
for year in [str(y) for y in [1984,2005,2006]]:
    thread = threading.Thread(target=retrieve_data, args=(year,))
    threads.append(thread)
    thread.start()
    time.sleep(1)

# Optionally, wait for all threads to complete
for thread in threads:
    thread.join()
